# Pull REALTIME Cryptocurrency Prices

In [1]:
import json
import websocket
# websockets là một thư viện để xây dựng máy chủ và máy khách WebSocket bằng Python
import pandas as pd

In [2]:
assets = ['BTCUSDT','ETHUSDT','BNBUSDT','DOGEUSDT','PEPEUSDT','XRPUSDT','SOLUSDT','AVAXUSDT','SHIBUSDT','ADAUSDT']

In [3]:
assets = [coin.lower() + '@kline_1m' for coin in assets]
assets


['btcusdt@kline_1m',
 'ethusdt@kline_1m',
 'bnbusdt@kline_1m',
 'dogeusdt@kline_1m',
 'pepeusdt@kline_1m',
 'xrpusdt@kline_1m',
 'solusdt@kline_1m',
 'avaxusdt@kline_1m',
 'shibusdt@kline_1m',
 'adausdt@kline_1m']

In [4]:
assets = '/'.join(assets)
assets
# Split the string by slashes and filter out empty strings
parts = assets.split('/')
filtered_parts = [''.join(part.split('/')) for part in assets.split('///') if part]

# Replace the required parts to match the desired format
formatted_assets = [f"{pair}@kline_1m@kline_1m" for pair in filtered_parts]

# Join the results with ' / ' separator
result = '/'.join(formatted_assets)

print(result)

btcusdt@kline_1methusdt@kline_1mbnbusdt@kline_1mdogeusdt@kline_1mpepeusdt@kline_1mxrpusdt@kline_1msolusdt@kline_1mavaxusdt@kline_1mshibusdt@kline_1madausdt@kline_1m@kline_1m@kline_1m


In [5]:
def manipulation (source):
    rel_data = source['data']['k']['c']
    evt_time = pd.to_datetime(source['data']['E'],unit='ms')
    df = pd.DataFrame(rel_data, columns=[source['data']['s']], index=[evt_time])
    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df


In [8]:
def on_message(ws, message): 
    global source
    source = message
    message = json.loads(message)
    manipulation(message)

In [16]:
source

'{"stream":"solusdt@kline_1m","data":{"e":"kline","E":1740464048796,"s":"SOLUSDT","k":{"t":1740464040000,"T":1740464099999,"s":"SOLUSDT","i":"1m","f":1096849266,"L":1096849536,"o":"140.08000000","c":"140.07000000","h":"140.13000000","l":"140.07000000","v":"181.51500000","n":271,"x":false,"q":"25429.15898000","V":"87.26000000","Q":"12225.43015000","B":"0"}}}'

# Cấu trúc dữ liệu:
stream: "bnbusdt@kline_1m"

Xác định luồng dữ liệu đang được truyền.
- "bnbusdt": Cặp giao dịch Binance Coin/USDT.
- "@kline_1m": Dữ liệu nến (candlestick) với khung thời gian 1 phút.
- data: Chứa dữ liệu cụ thể của nến.

# Chi tiết data:
## Thông tin chung của sự kiện
e: 'kline'
- Loại sự kiện, ở đây là dữ liệu nến (candlestick).

E: 1733997068108
- Thời gian sự kiện được gửi đi (epoch time tính bằng millisecond).

s: 'BNBUSDT'
- Cặp giao dịch (Binance Coin so với USDT).

# Thông tin nến (k):
t: 1733997060000
- Thời điểm bắt đầu của nến (epoch time tính bằng millisecond).

T: 1733997119999
- Thời điểm kết thúc của nến (epoch time tính bằng millisecond).

s: 'BNBUSDT'
- Cặp giao dịch (lặp lại thông tin từ trước).

i: '1m'
- Khung thời gian của nến (1 phút).

f: 891342969
- ID của giao dịch đầu tiên trong nến.

L: 891343185
- ID của giao dịch cuối cùng trong nến.

o: '724.82000000'
- Giá mở cửa (open price) của nến.

c: '724.55000000'
- Giá đóng cửa (close price) của nến.

h: '724.83000000'
- Giá cao nhất (high price) của nến.

l: '724.55000000'
- Giá thấp nhất (low price) của nến.

v: '27.51800000'
- Khối lượng giao dịch (volume) trong nến (số lượng BNB được giao dịch).

n: 217
- Số lượng giao dịch (number of trades) diễn ra trong nến.

# x: False: 
Cờ đánh dấu liệu nến đã đóng hay chưa:
True: Nến đã đóng.
False: Nến vẫn đang hình thành.

q: '19941.75665000'
- Khối lượng giao dịch (volume) tính theo đồng quote (USDT).

V: '16.51100000'
- Khối lượng giao dịch của các lệnh mua vào (tính theo đồng BNB).

Q: '11965.17897000'
- Khối lượng giao dịch của các lệnh mua vào (tính theo đồng quote - USDT).

B: '0'
- Giá trị "Best bid volume" tại thời điểm đó.


In [10]:
socket = "wss://stream.binance.com:9443/stream?streams=" + assets
socket

'wss://stream.binance.com:9443/stream?streams=btcusdt@kline_1m/ethusdt@kline_1m/bnbusdt@kline_1m/dogeusdt@kline_1m/pepeusdt@kline_1m/xrpusdt@kline_1m/solusdt@kline_1m/avaxusdt@kline_1m/shibusdt@kline_1m/adausdt@kline_1m'

In [11]:
ws = websocket.WebSocketApp(socket, on_message=on_message) 
# Đây là một lớp trong thư viện websocket-client dùng để quản lý kết nối WebSocket.
# Nó cho phép thiết lập kết nối và định nghĩa các hành vi cho các sự kiện như nhận tin nhắn (on_message), kết nối thành công (on_open), hoặc xảy ra lỗi (on_error).
ws.run_forever()

                timestamp  AVAXUSDT
0 2025-02-25 06:07:51.076     21.73
                timestamp   BTCUSDT
0 2025-02-25 06:07:52.019  91845.14
                timestamp  ETHUSDT
0 2025-02-25 06:07:52.019   2493.3
                timestamp  BNBUSDT
0 2025-02-25 06:07:52.109   609.22
                timestamp  PEPEUSDT
0 2025-02-25 06:07:52.245  0.000008
                timestamp  ADAUSDT
0 2025-02-25 06:07:52.310   0.6833
                timestamp  XRPUSDT
0 2025-02-25 06:07:52.324   2.2779
                timestamp  DOGEUSDT
0 2025-02-25 06:07:52.574   0.21109
                timestamp  SOLUSDT
0 2025-02-25 06:07:52.804   140.59
                timestamp  AVAXUSDT
0 2025-02-25 06:07:53.090     21.74
                timestamp  SHIBUSDT
0 2025-02-25 06:07:53.448  0.000014
                timestamp  ETHUSDT
0 2025-02-25 06:07:54.018   2493.3
                timestamp  PEPEUSDT
0 2025-02-25 06:07:54.246  0.000008
                timestamp  XRPUSDT
0 2025-02-25 06:07:54.318   2.2774
      

True

# Filter Data datetime and export file .csv

In [19]:
import requests
import pandas as pd
import time

# Define the assets
assets = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'DOGEUSDT', 'PEPEUSDT', 
          'XRPUSDT', 'SOLUSDT', 'AVAXUSDT', 'SHIBUSDT', 'ADAUSDT']

# Define the date range (UNIX timestamps in milliseconds)
start_date = int(pd.Timestamp("2024-01-01").timestamp() * 1000)
end_date = int(pd.Timestamp("2024-12-12").timestamp() * 1000)

# Function to fetch kline data from Binance API
def fetch_kline_data(symbol, interval='5m', start_time=None, end_time=None):
    base_url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "endTime": end_time,
        "limit": 1000  # Maximum records per request
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for {symbol}: {response.status_code}")
        return []

# Fetch data for all assets within the date range
all_data = []
for asset in assets:
    print(f"Fetching data for {asset}...")
    current_start = start_date
    while current_start < end_date:
        klines = fetch_kline_data(asset, start_time=current_start, end_time=end_date)
        if not klines:
            break
        for kline in klines:
            all_data.append({
                "symbol": asset,
                "timestamp": pd.to_datetime(kline[0], unit='ms'),
                "open": float(kline[1]),
                "high": float(kline[2]),
                "low": float(kline[3]),
                "close": float(kline[4]),
                "volume": float(kline[5]),
                # "Number of trades": float(kline[6]),
                # "Taker buy base asset volume": float(kline[7])
            })
        current_start = klines[-1][0] + 1  # Move to the next time window
        time.sleep(0.2)  # Avoid hitting rate limits

# Convert to DataFrame
df = pd.DataFrame(all_data)

# Filter the DataFrame by the date range (if needed for validation)
df = df[(df['timestamp'] >= "2024-09-01") & (df['timestamp'] <= "2024-12-12")]


Fetching data for BTCUSDT...
Fetching data for ETHUSDT...
Fetching data for BNBUSDT...
Fetching data for DOGEUSDT...
Fetching data for PEPEUSDT...
Fetching data for XRPUSDT...
Fetching data for SOLUSDT...
Fetching data for AVAXUSDT...
Fetching data for SHIBUSDT...
Fetching data for ADAUSDT...


In [13]:
# Lưu dữ liệu thành 10 tệp CSV riêng biệt
for asset in assets:
    df_asset = df[df["symbol"] == asset]
    file_name = f"{asset}.csv"
    df_asset.to_csv(file_name, index=False)
    print(f"Đã lưu {file_name}")

Đã lưu BTCUSDT.csv
Đã lưu ETHUSDT.csv
Đã lưu BNBUSDT.csv
Đã lưu DOGEUSDT.csv
Đã lưu PEPEUSDT.csv
Đã lưu XRPUSDT.csv
Đã lưu SOLUSDT.csv
Đã lưu AVAXUSDT.csv
Đã lưu SHIBUSDT.csv
Đã lưu ADAUSDT.csv



To adapt your existing Binance Kline data-fetching code into a robust system that incorporates AI-driven pattern detection and predictive insights, we can structure the development into several steps:

Step 1: Automate Detection of Chart Patterns Using AI

Preprocessing Data

In [14]:
# After the data fetching loop
df = pd.DataFrame(all_data)
if df.empty:
    print("No data fetched. Please check the API calls or data range.")
else:
    print("Data fetched successfully!")


Data fetched successfully!


In [17]:
def compute_bollinger_bands(series, window=20, num_std=2):
    sma = series.rolling(window=window).mean()
    std_dev = series.rolling(window=window).std()
    upper_band = sma + (num_std * std_dev)
    lower_band = sma - (num_std * std_dev)
    return upper_band, lower_band

df['Upper_Band'], df['Lower_Band'] = compute_bollinger_bands(df['close'])

def compute_rsi(series, window=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def compute_macd(series, fast=12, slow=26, signal=9):
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd  # Return only the MACD line; add signal_line if needed


In [18]:
print(df.columns)


Index(['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'Upper_Band', 'Lower_Band'],
      dtype='object')


In [ ]:

from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Feature Engineering Example
def add_technical_indicators(df):
    df['SMA_20'] = df['close'].rolling(window=20).mean()
    df['RSI'] = compute_rsi(df['close'], window=14)  # Implement compute_rsi
    df['MACD'] = compute_macd(df['close'])          # Implement compute_macd
    return df

df = add_technical_indicators(df)

# Resample to a desired time interval
def resample_data(df, interval='1H'):
    # Convert timestamp to datetime if not already
    if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
    df.set_index('timestamp', inplace=True)
    df_resampled = df.resample(interval).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna().reset_index()
    return df_resampled


In [ ]:
import psutil

# Get system memory usage
memory = psutil.virtual_memory()
print(f"Available Memory: {memory.available / (1024 ** 3):.2f} GB")


Available Memory: 11.04 GB


Using Transformer or CNNs

Step 1: Create Image-Like Representation for Time-Series Data


In [ ]:
import numpy as np

def create_image_representation(df, window_size=20):
    num_samples = len(df) - window_size
    X = np.zeros((num_samples, window_size, 5, 1))  # Pre-allocate memory
    
    # Fill the array with windowed data
    for i in range(num_samples):
        window = df[['open', 'high', 'low', 'close', 'volume']].iloc[i:i+window_size].values
        X[i] = window.reshape((window_size, 5, 1))  # Reshape each window to match CNN input

    return X


In [ ]:
def data_generator(df, window_size=20, batch_size=32):
    num_samples = len(df) - window_size
    while True:  # Infinite loop to keep generating batches
        for i in range(0, num_samples, batch_size):
            X_batch = []
            y_batch = []
            for j in range(i, i + batch_size):
                window = df[['open', 'high', 'low', 'close', 'volume']].iloc[j:j+window_size].values
                X_batch.append(window.reshape((window_size, 5, 1)))
                y_batch.append(df['target'].iloc[j])  # Assuming 'target' is your label column
            yield np.array(X_batch), np.array(y_batch)


In [ ]:
# Compute technical indicators more efficiently using numpy or vectorized operations
import numpy as np

def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Apply vectorized RSI calculation
df['RSI'] = compute_rsi(df['close'])


Step 2: Split the Data into Training and Testing Sets

In [ ]:
import numpy as np
import pandas as pd
# Sample a subset of the data if it's too large
df_sampled = df.sample(frac=0.1, random_state=42)  # Take 10% of the data for testing

def create_image_representation(df, window_size=20):
    """
    Convert time-series data into image-like representation for CNN.
    
    :param df: DataFrame with columns ['open', 'high', 'low', 'close', 'volume']
    :param window_size: Number of previous time-steps to create the image-like sequence
    :return: np.array with shape (num_samples, window_size, num_features, 1)
    """
    # Column names to be used
    features = ['open', 'high', 'low', 'close', 'volume']
    
    # Number of samples is based on available rows minus window_size
    num_samples = len(df) - window_size
    X = []

    # For each time step, create a window of `window_size` steps
    for i in range(num_samples):
        window = df[features].iloc[i:i+window_size].values
        X.append(window)
    
    # Convert the list to numpy array and reshape to CNN-compatible format
    X = np.array(X)
    X = X.reshape((X.shape[0], window_size, X.shape[2], 1))  # Add channel dimension
    
    return X

# Example usage:
# Assume df_resampled is a DataFrame with the necessary columns ['open', 'high', 'low', 'close', 'volume']
# Create image-like data from your time-series DataFrame (df_resampled)
X = create_image_representation(df, window_size=20)

# Define target labels (y) based on your problem, assuming binary classification here
y = np.random.randint(0, 2, size=(X.shape[0],))  # Example binary labels (0 or 1)
